# Dask Analysis

Dask + Analysis.


In [44]:
import glob

In [45]:
files = glob.glob("*.parquet")

In [46]:
for file in files:
    ...
    #dd.read_parquet(file)
file

'candump_0x421.parquet'

In [47]:
import dask
import dask.dataframe as dd

In [48]:
import os

In [49]:
dd.read_parquet(file)
lazy_results = list()
for file in files:
    task = dask.delayed(dd.read_parquet)(os.path.abspath(file))
    task = task.repartition(npartitions=10)
    lazy_results.append(task)

In [50]:
from dask.cache import Cache
cache = Cache(2e9)  # Leverage two gigabytes of memory
cache.register()    # Turn cache on globally
from dask.distributed import Client, progress
client = Client()
client

/opt/conda/lib/python3.8/site-packages/distributed/client.py:1129: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| lz4     | 3.1.0         | 3.0.2         | 3.0.2         |
| python  | 3.8.6.final.0 | 3.8.0.final.0 | 3.8.0.final.0 |
+---------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tcp://scheduler:8786 Dashboard: http://scheduler:8787/status,Cluster Workers: 1 Cores: 12 Memory: 33.48 GB


In [51]:
df_ = dask.delayed(dd.concat)(lazy_results).repartition(npartitions=8)

In [52]:
df = dask.compute(df_)
df2 = dask.persist(df)

In [53]:
df[0]

,timestamp,canbus,can_id,can_id_hex,can_data_hex,B00,b00,B01,b01,B02,b02,B03,b03,B04,b04,B05,b05,B06,b06,B07,b07
npartitions=8,,,,,,,,,,,,,,,,,,,,,
,float64,object,int64,object,object,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [54]:
d = df[0]

In [55]:
result = d.groupby("can_id_hex").agg({"timestamp": ["count"]})

In [56]:
r = result.compute()

In [57]:
r.sort_values(("timestamp", "count"))

,timestamp
,count
can_id_hex,
421,852
4F1,856
2A8,8542
2E8,8542
3E1,8542
3A3,8542
263,8542
3A4,8543


In [67]:
h421 = d[d.can_id_hex=="421"]

In [63]:
h421_2 = h421.groupby("can_data_hex").agg({"timestamp": ["count"]})

In [64]:
h421_2.compute()

,timestamp
,count
can_data_hex,
2C66A98802840000,3
2C66A98902840000,2
2C6AA98802840000,1
2D02AA8702840000,6
2D06AA8702840000,4
...,...
2EE6A98802840000,1
2EEAA98702840000,2


In [69]:
h421_3 = h421.groupby("B00").agg({"timestamp": ["count"]})

In [74]:
h421.groupby(['B00','B01','B02']).agg({"timestamp": ["count"]}).compute()

timestamp
                count
B00 B01 B02          
44  102 169         5
    106 169         1
45  2   170         6
    6   170         4
    10  170         2
    14  170         6
    18  170         1
    98  169         2
    102 169        14
    106 169        21
    110 169        20
    114 169        19
    118 169        22
    122 169        14
    126 169        28
    130 169        15
    134 169        16
    138 169        20
    142 169        17
    146 169        19
    150 169        21
    154 169         6
    158 169        25
    162 169        23
    166 169        16
    170 169        25
    174 169        23
    178 169        25
    182 169        12
    186 169        30
    190 169        18
    194 169        21
    198 169        27
    202 169        13
    206 169        29
    210 169         9
    214 169         3
    218 169        39
    222 169        12
    226 169        29
    230 169        13
    234 169        14
    238 169        24
    242 169        14
    246 169        13
46  2   170        26
    6   170        13
    14  170        29
    18  170        12
    226 169         1
    230 169         1
    234 169         2
    238 169         8
    242 169         8
    246 169        16